In [1]:
import pandas as pd
import numpy as np
import json
import csv
import pandas_profiling as pp
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
#Загрузка датасетов
#df = pd.read_csv('data/data.csv')
#test = pd.read_csv('data/test.csv')

In [4]:
from google.colab import drive
drive.mount('disk')

Drive already mounted at disk; to attempt to forcibly remount, call drive.mount("disk", force_remount=True).


In [5]:
df = pd.read_csv('/content/disk/MyDrive/Colab Notebooks/Проект 5/data.csv')
test = pd.read_csv('/content/disk/MyDrive/Colab Notebooks/Проект 5/test.csv')

Обзор данных

In [6]:
df.head(3)

,bodyType,brand,car_url,color,options,description,engineDisplacement,enginePower,fuelType,mileage,model_name,name,productionDate,vehicleTransmission,owners,pts,own,drive,wheel,price
0,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/outl...,белый,"{'Комфорт': ['Круиз-контроль', 'Бортовой компь...","Куплен в конце 12 года, японская сборка. Состо...",2.0,146.0,Бензин,112000,outlander,III,2012,вариатор,3 или более,Дубликат,NaN,полный,Левый,949000.0
1,микровэн,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/mini...,белый,{'Безопасность': ['Антипробуксовочная система ...,"2014год, октябрь.Без пробега по России. 68й ку...",41.0,30.0,Электро,78493,minicab,1999-2014,2014,автоматическая,1 владелец,Оригинал,4 месяца,задний,Правый,600000.0
2,хэтчбек 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/lanc...,зелёный,"{'Комфорт': ['Электростеклоподъёмники задние',...",.Ещё 3 дня РАСПРОДАЖА склада авто с пробегом! ...,1.8,143.0,Бензин,246525,lancer,X,2008,вариатор,3 или более,Дубликат,NaN,передний,Левый,365000.0


In [7]:
test.head(3)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,RUB,2014,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,RUB,2017,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,88000,2013,"{""code"":""SUPERB"",""name"":""Superb"",""ru_name"":""Су...",SUPERB,DSG 1.8 AMT (152 л.с.),5,1603226280,RUB,2014,1100658222,"{""id"":""20026323"",""nameplate"":""DSG"",""displaceme...",LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118285 entries, 0 to 118284
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   bodyType             118283 non-null  object 
 1   brand                118178 non-null  object 
 2   car_url              118178 non-null  object 
 3   color                118178 non-null  object 
 4   options              118178 non-null  object 
 5   description          117235 non-null  object 
 6   engineDisplacement   118149 non-null  object 
 7   enginePower          118149 non-null  float64
 8   fuelType             118149 non-null  object 
 9   mileage              118151 non-null  object 
 10  model_name           118151 non-null  object 
 11  name                 118131 non-null  object 
 12  productionDate       118151 non-null  object 
 13  vehicleTransmission  118149 non-null  object 
 14  owners               118151 non-null  object 
 15  pts              

In [10]:
#приведение названий колонок к одному имени
test = test.rename(columns={'Владельцы': 'owners', 'ПТС': 'pts', 'Владение': 'own', 'Привод':'drive', 'Руль':'wheel'})

In [ ]:
#обзор тестовый данный с помощью Profile Reportвыгружаем 
#результат выгружаем в pandas_profile_output.html
profile = pp.ProfileReport(test)
profile.to_file('pandas_profile_test_output.html')
profile = pp.ProfileReport(df)
profile.to_file('pandas_profile_data_output.html')

TypeError: ignored

После обзора данных выявлены некоторые различия в типах и формате данных в собранных и тестовых выборках. А так же отсутствие некоторых параметров в собранных данных, таких как страна производитель и год выпуска модели. Эти данные можно будет взять из тестовой выборки.
Так же требуют обработки и приведения к одномо виду данные о снаряжении автомобилей. 

Приведение признаков к одному виду

In [ ]:
#заведем списки для будующей групприровки данных по типам
cat_values = []
num_values = []
drop = []

In [ ]:
#bodyType - редкие значения переименуем в "редкий", категорийный признак, 16 уникальных значений
bodyType_counts = test.bodyType.value_counts()
def bType(b_type):
    if b_type in bodyType_counts:
        if bodyType_counts[b_type]<20:
            return 'редкий'
        else:
            return b_type
    return None
df.bodyType = df.bodyType.apply(bType)
test.bodyType = test.bodyType.apply(bType)
df.dropna(subset = ['bodyType'], inplace=True)
cat_values.append('bodyType')

In [ ]:
#brand - категорийный признак 12 уникальных значений.
cat_values.append('brand')

In [ ]:
#car_url - уникальные значения, для обучения будующей модели использоваться не будут.
drop.append('car_url')

In [ ]:
#color - категорийный признак 16 уникальных значений
cat_values.append('color')

In [ ]:
#complectation_dict - отсутствует в собранной выборке, очень больше колличество выпадов в тестовой выборке
drop.append('complectation_dict')

In [ ]:
#description - описание продавца, на первом этапе обучения модели данный признак использовать не будем,
#в дальнейшем создадим дополнительные признаки на его основе.
drop.append('description')

In [ ]:
#engineDisplacement - числовой признак, приведем к единому вещественному типу данных
test.engineDisplacement = test.engineDisplacement.apply(lambda x: (x[:-4]) if x[:-4] else '2.0')
cat_values.append('engineDisplacement')
#в тестовых данных есть выпады данных, заполним их средними значениями
test.fillna('2.0', inplace=True)

In [ ]:
#enginePower - числовой признак, приведем к единому целочисленному типу данных
test.enginePower = test.enginePower.apply(lambda x: int(x[:-4]) if x[:-4] else None)
num_values.append('enginePower')

In [ ]:
#equipment_dict - на его основе созданим признак со списком опций имеющихся у авто,
#в дальнейшем создадим отдельный признак по каждой опции
options = pd.read_csv("/content/disk/MyDrive/Colab Notebooks/Проект 5/options.csv", sep=";", encoding="cp1251")
options_dict = {}
for line in options.values:
    options_dict[line[1]] = line[0]
    
def options_translater(element):
    """Функция переводит данные по присутствующим опциям в собраном датесете и приводит их к списку"""
    elm = eval(element)
    data = []
    if type(elm) != type(dict()):
        return None        
    for option in elm.values():
        for item in option:
            if item in options_dict:
               data.append(options_dict[item])
    return data

def options_optimizer(element):
    """Фунция приводит данные по присутствующим опциям в тестовом датасете 
    к списку и к единому виду с данными в собранным датасетом"""
    try:
        elm = json.loads(element)
        if type(elm) != type(dict()):
          raise Exception
    except Exception:
        return None
    data = []
    for item in elm:
        if item in options_dict.values():
            data.append(item)
    return data

#корректировка и приведение данных 
df['equipment_dict'] = df.options.apply(options_translater)
test['equipment_dict'] = test['equipment_dict'].apply(options_optimizer)
drop.append('options')

In [ ]:
#fuelType - категорийный признак, необходимо привести к одному виду
def fuel_correct(element):
    if pd.isna(element):
        return None
    """функция приводит данные в собраном датасете к данным тестовой выборки"""
    el = element.lower()
    if element.find("газобаллонное оборудование") !=-1:
        el = 'газ'
    return el
df.fuelType = df.fuelType.apply(fuel_correct)
cat_values.append('fuelType')

In [ ]:
#image - признак отсутствует в собраной выборке, для обучения модели использоваться не будет
#model_info - признак отсутствует в собраной выборке, для обучения модели использоваться не будет
drop.append('image')
drop.append('model_info')

In [ ]:
#model_name - категорийный признак, необходимо привести к одному виду с отобранной выборкой
test.model_name = test.model_name.apply(str.lower) #переводим тестовые данные в нижный регистр
cat_values.append('model_name')

In [ ]:
#modelDate
modelName = test.groupby('model_name').modelDate.mean().index
modelDate = [int(x.round()) for x in test.groupby('model_name').modelDate.mean().values]
modelDateDict = dict(zip(modelName,modelDate))
df['modelDate'] = df.model_name.apply(lambda x: modelDateDict[x] if x in modelDateDict else None)
num_values.append('modelDate')

In [ ]:
#name - сводная информация присутствует в других признаках
drop.append('name')

In [ ]:
#numberOfDoors, parsing_unixtime - данные в собраном датасете отсутствуют
drop.append('numberOfDoors')

In [ ]:
#parsing_unixtime - данные будем использовать для приведения цен к ценам в момент парсинга
#парсинг тестовых данных проводился  с 20 Oct 2020 по 26 Oct 2020
#парсинг собранного датасета с 5 Apt 2021 по 10 Apt 2021
drop.append('parsing_unixtime')

In [ ]:
#priceCurrency - содержит единственное значение, для обучения модели использоваться не будет
drop.append('priceCurrency')

In [ ]:
#productionDate - дата производства - числовой признак, формат и тип данный собранных и тестовых данных совпадают
num_values.append('productionDate')

In [ ]:
#sell_id - id объявления для обучения модели использоваться не будет
#super_gen, vehicleConfiguration - признаки отсутствует в собранных данных
drop.append('sell_id')
drop.append('super_gen')
drop.append('vehicleConfiguration')

In [ ]:
#vehicleTransmission - категорийный признак, формат и тип данный собранных и тестовых данных совпадают
cat_values.append('vehicleTransmission')

In [ ]:
#vendor - категорийный признак, добавляем в собранные данные
#берем информацию из тестовых данных
vendors = {brand:'EUROPEAN' for brand in test[test.vendor == 'EUROPEAN'].brand.value_counts().index}
vendors.update({brand:'JAPANESE' for brand in test[test.vendor == 'JAPANESE'].brand.value_counts().index})
df['vendor'] = df.brand.apply(lambda x: vendors[x])
cat_values.append('vendor')

In [ ]:
#owners - категорийный признак, формат и тип данный собранных и тестовых данных совпадают
cat_values.append('owners')
#own - очень большое колличество выпадов как в тестовых так и отобранных данных
#cat_values.append('own')
drop.append('own')
   

In [ ]:
#pts - категорийный признак, формат и тип данный собранных и тестовых данных совпадают
cat_values.append('pts')
#убрем некоректные значения данного параметра
pts_types = {'Дубликат', 'Оригинал'}
test.pts = test.pts.apply(lambda x: x if x in pts_types else 'Оригинал')

In [ ]:
#drive - категорийный признак, формат и тип данный собранных и тестовых данных совпадают
cat_values.append('drive')  

In [ ]:
#wheel - категорийный признак, формат и тип данный собранных и тестовых данных совпадают
cat_values.append('wheel')   

In [ ]:
#Состояние, Таможня - признаки имеет единственное значение, для обучения модели применяться не будут
drop.append('Состояние')
drop.append('Таможня') 

Чистим и объединяем тестовый и собранный датафрейм

In [ ]:
#удаляем задвоенные строки
for_drop = df[df['car_url'].isin(df['car_url'].mode().values)].sort_values('car_url').index[::2]
df.drop(for_drop, inplace=True)

In [ ]:
#добавляем признак для разделения данных на тестовые и обучающие
df['test'] = 0
test['test'] = 1
data = pd.concat([df, test])

In [ ]:
#создаем признаки для опций
equipment = Counter()
bin_values = []
for options in data.equipment_dict:
    if options:
        for option in options:
            equipment[option] += 1
#на основе топ 150 опций создадим бинарные признаки 

def make_options(options):
    if options==None:
        return 0
    if item[0] in options:
        return 1
    else:
        return 0

for item in equipment.most_common(20):
    data['options_'+item[0]] = data.equipment_dict.apply(make_options)
    bin_values.append(item[0])
drop.append('equipment_dict')

In [ ]:
#удаляем признаки, которые не будут использоваться в обучение модели
data.drop(drop, inplace=True, axis=1)

In [ ]:
#заполняем и чистим пропуски пропуски
data.price.fillna(0, inplace=True)
data.dropna(inplace=True)

In [ ]:
data[num_values].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150080 entries, 0 to 34685
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   enginePower     150080 non-null  float64
 1   modelDate       150080 non-null  float64
 2   productionDate  150080 non-null  object 
dtypes: float64(2), object(1)
memory usage: 4.6+ MB


In [ ]:
# Переводим признаки из float в int (иначе catboost выдает ошибку)
for feature in [num_values]:
  data[feature]=data[feature].astype('int32')
    

In [ ]:
profile = pp.ProfileReport(data)
profile.to_file('/content/disk/MyDrive/Colab Notebooks/Проект 5/pandas_profile_data_output.html')

TypeError: ignored

CatBoost


In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
# Setup
VAL_SIZE   = 0.33   # 33%
N_FOLDS    = 5

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor
# CATBOOST
ITERATIONS = 2000
LR         = 0.1

In [ ]:
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )

In [ ]:
#делим выборку на тестовую и обучающую
X_sub = data[data.test == 1].drop(['price'], axis=1,)
X = data[data.test == 0].drop(['price'], axis=1,)
y = data[data.test == 0].price.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
X_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34686 entries, 0 to 34685
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   bodyType                      34686 non-null  object
 1   brand                         34686 non-null  object
 2   color                         34686 non-null  object
 3   engineDisplacement            34686 non-null  object
 4   enginePower                   34686 non-null  int32 
 5   fuelType                      34686 non-null  object
 6   mileage                       34686 non-null  object
 7   model_name                    34686 non-null  object
 8   productionDate                34686 non-null  int32 
 9   vehicleTransmission           34686 non-null  object
 10  owners                        34686 non-null  object
 11  pts                           34686 non-null  object
 12  drive                         34686 non-null  object
 13  wheel           

In [ ]:
model.fit(X_train, y_train,
         cat_features=cat_values,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.3178436	test: 1.3248571	best: 1.3248571 (0)	total: 220ms	remaining: 7m 20s
100:	learn: 0.1358381	test: 0.1369121	best: 0.1369121 (100)	total: 15s	remaining: 4m 41s
200:	learn: 0.1040299	test: 0.1049191	best: 0.1049191 (200)	total: 29.9s	remaining: 4m 27s
300:	learn: 0.0923178	test: 0.0930627	best: 0.0930627 (300)	total: 45s	remaining: 4m 13s
400:	learn: 0.0845552	test: 0.0850731	best: 0.0850731 (400)	total: 1m	remaining: 4m
500:	learn: 0.0784927	test: 0.0789736	best: 0.0789736 (500)	total: 1m 16s	remaining: 3m 48s
600:	learn: 0.0738004	test: 0.0744916	best: 0.0744916 (600)	total: 1m 32s	remaining: 3m 35s
700:	learn: 0.0701854	test: 0.0712883	best: 0.0712883 (700)	total: 1m 49s	remaining: 3m 22s
800:	learn: 0.0675120	test: 0.0689800	best: 0.0689800 (800)	total: 2m 5s	remaining: 3m 8s
900:	learn: 0.0651112	test: 0.0668683	best: 0.0668683 (900)	total: 2m 22s	remaining: 2m 54s
1000:	learn: 0.0629063	test: 0.0650144	best: 0.0650144 (1000)	total: 2m 40s	remaining: 2m 40s
1100:	le

Submission

In [ ]:
predict_submission = model.predict(X_sub)
predict_submission

array([ 493959.9624689 , 1269203.46225384,  874807.02116801, ...,
        387847.61627103, 1350101.48268078, 1307004.01672805])